### 다음 데이터 크롤링 (2021-03-12)
### 2. 웹툰별 회차별 썸네일 크롤링

In [7]:
from IPython.display import clear_output # clear_output() 으로 아웃풋 제거 가능
# 값 연속적으로 모두 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import pandas as pd
import requests

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
data = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/daum_webtoon/D_Webtoon_1_toonsinfo.csv")

In [ ]:
# data

In [ ]:
# # 우두커니 웹툰의 경우
# url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ df.loc[0]['nickname']
# temp_df= get_data(url3)

# temp_df['data']['webtoon']['webtoonEpisodes'][0].keys()

dict_keys(['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable', 'ageGrade'])

In [ ]:
# wbt_chapter = pd.DataFrame(columns=['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable'])
# df_ch = temp_df['data']['webtoon']
# for data in df_ch['webtoonEpisodes']:
#     wbt_chapter.loc[len(wbt_chapter)] = data
# # wbt_chapter.tail()

In [ ]:
# wbt2_df=wbt_chapter[['id', 'episode', 'title', 'thumbnailImage']]

In [ ]:
# wbt2_df2 = wbt2_df[['id', 'episode', 'title']].copy()

# img_url = []
# for i, data in wbt2_df.iterrows():
#     try:
#         img_url.append(data['thumbnailImage']['url'])
#     except:
#         img_url.append(None)

# wbt2_df2['img_url'] = pd.Series(img_url)
# wbt2_df2

## 본격 크롤링

In [19]:
def getChapterImg(ls_url_nicks):
    print('Getting webtoon`s chapter list from Daum...')

    headers = {'Content-Type': 'text/html'}
    cookies = {'session_id' : 'RECENT_USE=on; RECENT_SEARCH=%EC%8C%8D%EA%B0%91%2C; RECENT_VIEW=105701|54701|104995|50615|90827; SLEVEL=0; ALARM_COUPON=20210315192117; __T_=1; VOTE=MTAuNTE%3D; MY=MTAuNTE%3D; ORDER=MTAuNTE%3D; HM_CU=5DRHdXDNCXg; PROF=0603012032024076024120UiQPJk7X-6w0mlxoempuua-WtEc2RT95vdkDQkloQ8t.KuZP3z7wEl2bxriNh5SdkQ00LYYSA9A1_cGNLCyhCzrwOjTi9JCLCPIz3mlNTt7zkh799Fe9fQCTuFahgZwN7arRuq1D_iaAuoc0fIauQlo8OTLcCscrxJX8Uw00rfhTzfRpBVHPn23B_vTzGDH5t.ow35ZYntxYtQOcleZXKh2-G69UsZ5Joi8PC77SepzPx_EoF25QUPUbuh6eoCQZjRxg2d7oLS_CliiS.l83vhtpaXRyxA00; AGEN=4ywGKJy_PaRaCTDlmS0HW-WYwYc1j29NCjAYA-_LArc; TIARA=-73vHQQZIRxfdXAvj44dQwwavwjLg6nvVGXEtrCIlZXu863u25mi8mC7Y9bO.h-EsfrP1HeF.yELGzHaPO31P81NuBns.l5W; TS=1615950634; HTS=bVfL1dMfBsVxzsj8gVzI5g00; LSID=504abe8c-6e68-42db-bbd8-2af123c1d5c21615950634905; _fbp=fb.1.1615950639787.830177619; _T_ANO=OKZ+Q7Y4USXeyLVg1FxHtaULMyth/41zPwjH70cY9uaj3fZxApYc3Gyg9+qIuDbYCMYz4PErFebItCEXrE4FgQxYjtA9vUB2VsgDa0sWRapm8NuIqTsv7//6qVc8QDmjPW50kHWVY7ace5tdJ4Cli3uJ09HbrYl/Av/2SD/oo+eXt4qt+GHkqWS1/Mkch0h5ZNzmgcXcfsekrvv6S76JrB/DznWm5G+2pA+QUGK/LUgeLB0qQ8shpQI0MBB9O8cFE6Kp9X1kLiaSbFfgJkYuUgCEYVEO1+Xvcd29B7wC0wzohhBaiCZQCTcbh4DgEb7fd9Uxb3qr7RicroD5HEdUCw=='}

    df = pd.DataFrame(columns=['wbt_id','wbt_nkname', 'wbt_name','id', 'episode', 'title', 'img_url'])
    
    cnt = 0
    error = 0
    error_ls = []
    
    for nickname in ls_url_nicks:
        try :
            url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ nickname
            response = requests.get(url3, headers=headers, cookies=cookies) # 세션을 유지하면서 url가져오기
        except:
            print("server rejected please wait 45sec...")
            time.sleep(45)
            print("now reconnect again...", end="")
            url3 = 'http://webtoon.daum.net/data/pc/webtoon/view/'+ nickname
            response = requests.get(url3, headers=headers, cookies=cookies) # 세션을 유지하면서 url가져오기
            print("success!!")
        try:        
            temp_df = response.json() # json -> dict 형식으로 알아서 바꿔줌

            wbt_chapter = pd.DataFrame(columns=['id', 'episode', 'title', 'shortTitle', 'thumbnailImage', 'episodeImage', 'encryptUseYn', 'serviceStatus', 'articleId', 'commentUseYn', 'dateCreated', 'webtoon', 'serviceType', 'multiType', 'multiBgm', 'multiBackgroundImage', 'price', 'padtoonImage', 'voteTarget', 'shareVoteTarget', 'isTopRecommend', 'simpleUrl', 'specialSearchString', 'specialSearchUrl', 'previewEndDate', 'isPaid', 'payWebtoonEpisode', 'product', 'ticketAvailable'])

            df_ch = temp_df['data']['webtoon']
            
            
            for data in df_ch['webtoonEpisodes']:
                wbt_chapter.loc[len(wbt_chapter)] = data
            wbt_chapter['wbt_id']=df_ch['id']
            wbt_chapter['wbt_nkname']=df_ch['nickname']
            wbt_chapter['wbt_name']=df_ch['title']

            wbt2_df=wbt_chapter[['wbt_id','wbt_nkname', 'wbt_name', 'id', 'episode', 'title', 'thumbnailImage']]
            wbt2_df2 = wbt2_df[['wbt_id','wbt_nkname', 'wbt_name', 'id', 'episode', 'title']].copy()
            
            img_url = []
            for i, data in wbt2_df.iterrows():
                try:
                    img_url.append(data['thumbnailImage']['url'])
                except:
                    img_url.append(None)

            wbt2_df2['img_url'] = pd.Series(img_url)

            df = df.append(wbt2_df2)

            if cnt % 20 == 0:
                print(cnt,"(",nickname,")..",end="")
            cnt += 1    
            
        except: # 에러발생시킨 웹툰 캐치
            
            error += 1
            print("error occurd!!(",error,'-',nickname,")..", end="")
            error_ls.append(nickname)
        
        
    return df.reset_index(drop=True), error_ls

In [20]:
df2,error_ls = getChapterImg(list(data['nickname']))
error_ls

Getting webtoon`s chapter list from Daum...
0 ( vacantly )..20 ( doggaebi )..40 ( yulmuartari )..60 ( disposableactor )..80 ( cubee )..100 ( BreakerDrawing )..120 ( godstory )..140 ( 4thgrade )..160 ( hellTemple )..180 ( notstopping )..200 ( homemaker )..220 ( kimong_fairytale )..240 ( 11yearafterour )..260 ( blackbehemoth )..280 ( hwaja )..300 ( godstime )..320 ( amoment )..340 ( rainbowfish )..360 ( eroro )..380 ( dings )..400 ( serotonin )..420 ( illang )..440 ( runningwoman )..460 ( RTX )..480 ( simbongsa )..500 ( sangod )..520 ( foolforyou )..540 ( mhz )..560 ( myhomebird )..580 ( maidnvampire )..600 ( intersection )..620 ( UNDERCITY2 )..640 ( groomingsunday )..660 ( afternoonheros )..680 ( moase )..700 ( allinone )..720 ( vampiregirl )..740 ( goodrobot )..760 ( misosikga )..780 ( daddy2 )..800 ( happyparadise )..820 ( homelessblues )..error occurd!!( 1 - killmywife )..error occurd!!( 2 - todayandtoday )..840 ( gone )..860 ( wave )..error occurd!!( 3 - daddygoround )..880 ( jealou

['killmywife', 'todayandtoday', 'daddygoround']

In [21]:
df2

,wbt_id,wbt_nkname,wbt_name,id,episode,title,img_url
0,1444,vacantly,우두커니,50196,1,프롤로그,http://t1.daumcdn.net/webtoon/op/d0634b71310c2...
1,1444,vacantly,우두커니,50197,2,1화 이삿날,http://t1.daumcdn.net/webtoon/op/2b9b44f47e8df...
2,1444,vacantly,우두커니,50198,3,2화 꿈이겠지,http://t1.daumcdn.net/webtoon/op/87beb2a0deb96...
3,1444,vacantly,우두커니,50199,4,3화 폭우,http://t1.daumcdn.net/webtoon/op/fad8f36885f49...
4,1444,vacantly,우두커니,50322,5,4화 아버지의 그림,http://t1.daumcdn.net/webtoon/op/88c19843941d6...
...,...,...,...,...,...,...,...
79483,1222,gochoo,여기 고수 추가요!,34174,22,20화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
79484,1222,gochoo,여기 고수 추가요!,34285,23,21화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
79485,1222,gochoo,여기 고수 추가요!,34398,24,22화,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...
79486,1222,gochoo,여기 고수 추가요!,34494,25,에필로그,http://i1.cartoon.daumcdn.net/svc/image/U03/ca...


In [22]:
df2.to_csv("D_Webtoon_2_thumbnail.csv",index=False)